In [36]:
%run Classes/SPDEs.ipynb
%run Classes/Rule.ipynb
%run Classes/Model.ipynb
%run Classes/Noise.ipynb
%run Classes/full_visualization.ipynb
%run Algorithms/Algorithm_2.ipynb
%run Classes/Burgers_PDE_FIND.ipynb

#Disable progress bars from tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [45]:
#set global parameters
a, b, s, t = -8, 8, 0, 10 # space-time boundaries

X_, T_ = np.linspace(a,b,513), np.linspace(s,t,1001) # space-time grid for generating solutions
X, T = X_[::2], T_[::10] # space-time grid of observed points. O_X and O_T

eps = 0.2 #Viscosity

k = 15 # Number of realizations (multiple of 3)
m = 5 # Number of test cases, m < k

N = 2 # Number of experiments

#Model parameters
height = 3
deg = 2.5
kernel_deg = 2
noise_deg = -1.5
free_num = 2
BC = 'P'

R = Rule(kernel_deg = kernel_deg,
         noise_deg = noise_deg,
         free_num = free_num) # initialize rule. No multiplicative width and forcing. 

#Polynomial interpolation parameters for PDE-FIND
deg_x = 1
deg_t = 1

time_scale = 10 # increase the time interval for finer Euler scheme in PDE-FIND

In [48]:
%%time

exps_Alg2 = []
exps_v = []
exps_v_coef = []
exps_find = []
exps_find_eps =[]
exps_find_lambd =[]

for n in range(0,N):
    # Create realizations of initial conditions. 
    IC1 = Noise().initial(k//3, X_, scaling = 1) # one cycle
    IC2 = Noise().initial(k//3, X_, scaling = 2) # two cycles
    IC4 = Noise().initial(k//3, X_, scaling = 4) # four cycles

    IC = np.concatenate([IC1, IC2, IC4])

    W = np.zeros((k, len(T_), len(X_))) # No forcing
    
    #Generate solutions
    Burgers_ = SPDE(BC = BC, IC = IC, eps = eps).Burgers(W, T = T_, X = X_) # solve Burger's equations on the fine grid

    Burgers = Burgers_[:,::10,::2] # Observed points of Burger's equations
    
    Burgers = Burgers.astype('float32')
    #
    test = np.random.choice(np.arange(k), m, replace = False) # choose test set
    
    
    #ALGORITHM 2

    experiment = IML(Burgers, Rule = R, eps = eps, height = height, deg = deg, T = T, X = X) # initialize experiment for algorithm 2
    experiment.set_train_test(test = test) # initialize test set

    experiment.set_trees() # extract trees

    experiment.initialize_Predictions() 

    experiment.training_models() # Create models for Step 2 of Algorithm 2
    
    experiment.fit_training_model() # Linear fit for Step 2 of Algorithm 2
    
    experiment.learn_with_fitted() # Recurcive steps 3 & 4 of Algorithm 2
    
    # Translate prediction to a dataframe for the visualization class
    Prediction = experiment.to_df_list()

    # Translate solution to a dataframe for the visualization class
    Burgers_true = [pd.DataFrame(B, index = T, columns = X) for B in Burgers]
    
    # Errors

    show = summary(experiment.test)
    er = show.errors(Burgers_true, Prediction, show = False)
    exps_Alg2.append(show)
    
    
    #ESTIMATE VISCOSITY
    
    # Regression to determine viscosity from train cases

    j = 0

    u_t = np.array([(Burgers_true[i].iloc[j+1].values-Burgers_true[i].iloc[j]).values/(T[1]-T[0]) for i in range(k) if i not in test]).flatten()

    dx = X[1]-X[0]

    M = SPDE(BC = 'P').Parabolic_Matrix(len(X)-1, 1, dx, inverse = False)

    u = np.array([Burgers_true[i].iloc[j].values for i in range(k) if i not in test])
    u_xx = np.array([np.dot(M, U) for U in u]).flatten()

    visc = lin_reg().fit(u_xx.reshape(-1, 1), u_t)
    exps_v_coef.append(visc.coef_)

    # Algorithm 2 on the true data with learnt viscosity eps

    experiment_v = IML(Burgers, Rule = R, eps = visc.coef_, height = height, deg = deg, T = T, X = X) # initialize experiment for algorithm 2
    experiment_v.set_train_test(test = test) # initialize test set

    experiment_v.trees = experiment.trees # extract trees

    experiment_v.initialize_Predictions() 

    experiment_v.training_models() # Create models for Step 2 of Algorithm 2
    
    experiment_v.fit_training_model() # Linear fit for Step 2 of Algorithm 2
    experiment_v.learn_with_fitted() # Recurcive steps 3 & 4 of Algorithm 2
    
    Prediction_v = experiment_v.to_df_list()

    show_v = summary(experiment.test)
    er_find = show_v.errors(Burgers_true,Prediction_v, show = False)
    exps_v.append(show_v)
    
    
    #PDE-FIND
    
    # PDE-FIND to learn coefficients of equation

    dt, dx = T[1]-T[0], X[1]-X[0]
    u = np.array([Burgers_true[i] for i in range(len(Burgers_true)) if i not in test])
    # get viscosity and non linear coeficient of the Burgers equation
    eps_find, lambd_find = PDE_FIND_Burgers(u, dt, dx, diff='finite_diff')
    """eps_find, lambd_find = PDE_FIND_Burgers(u,
                                            dt,
                                            dx,
                                            diff='poly',
                                            offset_x = 30,
                                            offset_t = 30,
                                            deg_x = deg_x,
                                            deg_t = deg_t)"""
    
    #IC_ = IC[:,::2]
    IC_ = np.array([B[0] for B in Burgers])

    W_ = np.zeros((k, len(T_), len(X)))

    Burgers_find = SPDE(BC = BC, IC = IC_, eps = 0.2).Burgers(W_, lambd = -1.0, T = T_, X = X)

    Burgers_find = [pd.DataFrame(Burgers_find[i, ::10, :], index = T, columns = X) for i in test]
    #Burgers_find = [pd.DataFrame(Burgers_find[i], index = T, columns = X) for i in test]
    
    show_find = summary(experiment.test)
    er_find = show_find.errors(Burgers_true, Burgers_find, show = False)
    exps_find.append(show_find)
    exps_find_eps.append(eps_find)
    exps_find_lambd.append(lambd_find)


Creating Model
Creating Model
Creating Model
Creating Model
CPU times: user 4min 36s, sys: 20.9 s, total: 4min 56s
Wall time: 37.5 s


In [49]:
for exps in [exps_Alg2, exps_v, exps_find]:

    print("Average average error:", np.mean([e.av_er for e in exps]))
    print("Maximum average error:", max([e.av_er for e in exps]))
    print("Minimum average error:", min([e.av_er for e in exps]))
    print('')

    print("Average maximum error:", np.mean([e.max_er for e in exps]))
    print("Maximum maximum error:", max([e.max_er for e in exps]))
    print("Minimum maximum error:", min([e.max_er for e in exps]))
    print('')

    print("Average minimum error:", np.mean([e.min_er for e in exps]))
    print("Maximum minimum error:", max([e.min_er for e in exps]))
    print("Minimum minimum error:", min([e.min_er for e in exps]))
    print('')

    print("=====================")

Average average error: 3.82887931367362
Maximum average error: 6.885345777763925
Minimum average error: 0.7724128495833149

Average maximum error: 15.223245756165475
Maximum maximum error: 26.693696148230444
Minimum maximum error: 3.752795364100508

Average minimum error: 0.010305938353728984
Maximum minimum error: 0.014448637337467594
Minimum minimum error: 0.006163239369990373

Average average error: 4.380397224911535
Maximum average error: 6.991313716831205
Minimum average error: 1.769480732991865

Average maximum error: 16.051627482504152
Maximum maximum error: 27.208071574629056
Minimum maximum error: 4.895183390379245

Average minimum error: 0.010151031436081214
Maximum minimum error: 0.014572959345982642
Minimum minimum error: 0.005729103526179787

Average average error: 0.13631140253766397
Maximum average error: 0.15498548585804647
Minimum average error: 0.11763731921728145

Average maximum error: 0.3007772479935068
Maximum maximum error: 0.3218615132446336
Minimum maximum erro

In [29]:
print("Average viscosity:", np.mean(exps_v_coef))
print("Maximum viscosity:", max(exps_v_coef))
print("Minimum viscosity:", min(exps_v_coef))

Average viscosity: 0.11502342457769954
Maximum viscosity: [0.11562856]
Minimum viscosity: [0.11441829]


In [41]:
print("Average viscosity: {}, lambda: {}".format(np.mean(exps_find_eps), np.mean(exps_find_lambd)))
print("Maximum viscosity: {}, lambda: {}".format(max(exps_find_eps), max(exps_find_lambd)))
print("Minimum viscosity: {}, lambda: {}".format(min(exps_find_eps), min(exps_find_lambd)))

Average viscosity: 0.14115729966824053, lambda: -0.9435499193213398
Maximum viscosity: 0.1413886093206219, lambda: -0.9398373499404485
Minimum viscosity: 0.1409259900158592, lambda: -0.9472624887022311


In [42]:
exps_find[0].av_er

0.10007642566055246

In [43]:
exps_find[1].av_er

0.26945851415879074

In [44]:
exps_find[2].av_er

IndexError: list index out of range